In [29]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder, StandardScaler

In [30]:
dataset = pd.read_csv("credit_simple.csv", sep=";")
dataset.shape

(1000, 8)

In [31]:
dataset.head()

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO,CLASSE
0,1169.0,4,67,nenhum,01/01/2019,masculino solteiro,radio/tv,bom
1,5951.0,2,22,nenhum,01/01/2020,fem div/cas,radio/tv,ruim
2,2096.0,3,49,nenhum,02/01/2020,masculino solteiro,educação,bom
3,7882.0,4,45,nenhum,02/01/2019,masculino solteiro,mobilia/equipamento,bom
4,4870.0,4,53,nenhum,03/01/2018,masculino solteiro,carro novo,ruim


In [32]:
# separando variável dependente das variáveis independentes
y = dataset['CLASSE']
X = dataset.iloc[:, :-1]

In [33]:
X

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO
0,1169.0,4,67,nenhum,01/01/2019,masculino solteiro,radio/tv
1,5951.0,2,22,nenhum,01/01/2020,fem div/cas,radio/tv
2,2096.0,3,49,nenhum,02/01/2020,masculino solteiro,educação
3,7882.0,4,45,nenhum,02/01/2019,masculino solteiro,mobilia/equipamento
4,4870.0,4,53,nenhum,03/01/2018,masculino solteiro,carro novo
...,...,...,...,...,...,...,...
995,1736.0,4,31,nenhum,29/06/2018,fem div/cas,mobilia/equipamento
996,3857.0,4,40,nenhum,30/06/2018,masculino div/sep,carro usado
997,804.0,4,38,nenhum,03/07/2018,masculino solteiro,radio/tv
998,1845.0,4,23,nenhum,04/07/2019,masculino solteiro,radio/tv


### Valores Nulos

In [34]:
# verificando valores nulos
X.isnull().sum()

SALDO_ATUAL         7
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         8
PROPOSITO           0
dtype: int64

In [35]:
# saldo atual -> valor numérico: preenchendo com mediana
mediana = X['SALDO_ATUAL'].median()
mediana

np.float64(2323.0)

In [36]:
# preenchendo os valores de saldo atual
X['SALDO_ATUAL'].fillna(mediana, inplace=True)
X.isnull().sum()

C:\Users\Rogerio\AppData\Local\Temp\ipykernel_33976\2981516426.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  X['SALDO_ATUAL'].fillna(mediana, inplace=True)


SALDO_ATUAL         0
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         8
PROPOSITO           0
dtype: int64

In [37]:
# verificando a moda de estado civil
agrupado = X.groupby(['ESTADOCIVIL']).size()
agrupado

ESTADOCIVIL
fem div/cas               308
masculino casado/viuvo     92
masculino div/sep          50
masculino solteiro        542
dtype: int64

In [40]:
# preenchendo os valores de estado civil c/ moda
X['ESTADOCIVIL'].fillna('masculino solteiro', inplace=True)
X.isnull().sum()

SALDO_ATUAL         0
RESIDENCIADESDE     0
IDADE               0
OUTROSPLANOSPGTO    0
DATA                0
ESTADOCIVIL         0
PROPOSITO           0
dtype: int64

### Outliers

In [41]:
desvpad = X['SALDO_ATUAL'].std()
desvpad

np.float64(685936688.9820067)

In [ ]:
# verificando onde há outliers
X.loc[X['SALDO_ATUAL'] >= 2 * desvpad, 'SALDO_ATUAL']

127    2.541111e+09
160    2.154441e+10
Name: SALDO_ATUAL, dtype: float64

In [43]:
mediana = X['SALDO_ATUAL'].median()
mediana

np.float64(2323.0)

In [46]:
# eliminando os outliers (nesse caso, preenchendo com a mediana)
X.loc[X['SALDO_ATUAL'] >= 2 * desvpad, 'SALDO_ATUAL'] = mediana
X.loc[X['SALDO_ATUAL'] >= 2 * desvpad]

,SALDO_ATUAL,RESIDENCIADESDE,IDADE,OUTROSPLANOSPGTO,DATA,ESTADOCIVIL,PROPOSITO
